In [1]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276
#3) https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?Expires=1585601065&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=Lnpf6wT8rkozSh9av7U9nGuC7WAH6KuI2Cj3Y7G366gkGlh8D-Ie1Kc~TyBAUu~uMsVltleJcSv3p6TCm6HdFnhpyoTgLcYh6eFfvQwIUqbk1Bf4JZldgB~BDKUOwY1G0pA-HoKjvIAu3avO98SMO35upakm9OEBByd4nC9aXsjKRThd6bTpq1qIuuD9gh1l5FaM6hNRB0c2lCf4Q3adx7C3FW0NMwdWhcuF45A9f~dO3zTWWSQamoo5Otc-PHMMt96TetNcML~jy9ghgJeCPY6DJLUIwQAt03fENBluS~TjTJ17WD~n51xiRofb94fEJHoRHh0d-430LLwr7BX4IA__

In [2]:
import numpy as np
import os
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
import matplotlib.pyplot as plt
#import seaborn as sn
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import lfilter
import librosa
import pysptk
import math
import sys

#settings
import config

## Reading recordings

In [3]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [4]:
#Read wav data set

if config.featExtr_skip is False:
    print("Readings wavs...")

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(config.FOLDER_PATH) if os.path.isfile(os.path.join(config.FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = config.FOLDER_PATH+file_name
        print(full_fname)
    
        # load audio
        s = AudioSegment.from_wav(full_fname)
        print(full_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(full_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        #print(label)
        all_label.append(label)

Readings wavs...
data/YT_set/edited_wavs/edit_Wet Throat Infection Cough-tfc5cXiXMDc-Wet.wav
data/YT_set/edited_wavs/edit_Wet Throat Infection Cough-tfc5cXiXMDc-Wet.wav
data/YT_set/edited_wavs/edit_Heavy cold and sore throat coughing.-NaOVmYoIjbs-Dry.wav
data/YT_set/edited_wavs/edit_Heavy cold and sore throat coughing.-NaOVmYoIjbs-Dry.wav
data/YT_set/edited_wavs/edit_Spring Cold Coughing 2-AQOeIVbhFm4-Dry.wav
data/YT_set/edited_wavs/edit_Spring Cold Coughing 2-AQOeIVbhFm4-Dry.wav
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_# 34 coughing up crap again-rkF_uMizqoc-Wet.wav
data/YT_set/edited_wavs/edit_# 34 coughing up crap again-rkF_uMizqoc-Wet.wav
data/YT_set/edited_wavs/edit_# 30 Chesty and wet cough-

Listening to some of the audios

In [5]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Dry')

In [6]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Wet')

In [7]:
if config.featExtr_skip is False:
    s=all_s[15]
    s

## Feature extraction

In [8]:
import featureExtractionFunctions as featExtrLib

if config.featExtr_skip is False:

    feats = featExtrLib.feature_extraction_Step(all_s,all_id,all_label)
    
       

High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass 

## Load  (or store) features 

In [9]:
feats_fname = 'feats_df.pkl'

if config.featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [10]:
feats2 = featExtrLib.processingNaNvalues(feats)

In [11]:
label_dict = featExtrLib.createLabelDict_addLabel2df(feats2)
mean_std_feats = featExtrLib.frame_mean_std_chunk_modeling (feats2,label_dict)

## Model training

In [13]:
import modelTrainingFunctions as modelTrainLib

In [14]:
X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']

#ID_train.size

classifier 1:

In [15]:
from sklearn.linear_model import SGDClassifier

In [16]:
cls=SGDClassifier(loss='log', penalty='elasticnet',random_state = 1)

evaluate:

In [17]:
%time pred_probs = modelTrainLib.modelTraining(cls,X_train,y_train,ID_train)

CPU times: user 749 ms, sys: 12.3 ms, total: 761 ms
Wall time: 781 ms


In [18]:
pred_probs

,ID,Dry,Wet
0,1UDFq2InljM,3.037764e-05,9.999696e-01
1,1UDFq2InljM,2.040028e-07,9.999998e-01
2,1UDFq2InljM,9.997861e-01,2.139390e-04
3,1UDFq2InljM,9.999999e-01,1.015363e-07
4,1UDFq2InljM,1.755805e-02,9.824420e-01
...,...,...,...
460,zjd4HrJbc8o,8.609762e-01,1.390238e-01
461,zjd4HrJbc8o,9.825873e-01,1.741274e-02
462,zjd4HrJbc8o,6.824670e-02,9.317533e-01
463,zjd4HrJbc8o,9.999877e-01,1.232369e-05


In [ ]:
mean_pred_probs_1 = modelTrainLib.get_predClass_per_audio(pred_probs, label_dict)

classifier 2:

In [19]:
import GPy

In [20]:
class GP_mini():
    
    def __init__(self, kernel=None):
        self.kernel=kernel
        
    def set_default_kernel(self, dim):
        self.kernel=np.sum([GPy.kern.RBF(input_dim=1,active_dims=[ind]) for ind in range(dim)])
    
    def fit(self, xtrain, ytrain):
        
        # data conversion
        labels,names=pd.factorize(ytrain)
        self.classes_=names
        
        # check kernel
        if self.kernel is None: self.set_default_kernel(xtrain.shape[1])
        
        # train classifier
        self.model=GPy.models.GPClassification(xtrain,labels.reshape(-1,1),kernel=self.kernel.copy())
        self.model.optimize()
        
    def predict_proba(self, xtest):
        prob=self.model.predict(np.atleast_2d(xtest))[0]
        return np.concatenate((1-prob,prob),axis=1)

In [21]:
cls=GP_mini()

In [22]:
%time pred_probs = modelTrainLib.modelTraining(cls,X_train,y_train,ID_train)

CPU times: user 7h 31min 43s, sys: 1h 3min 38s, total: 8h 35min 21s
Wall time: 4h 44min 39s


In [23]:
pred_probs

,ID,Wet,Dry
0,1UDFq2InljM,0.376311,0.623689
1,1UDFq2InljM,0.929443,0.070557
2,1UDFq2InljM,0.587794,0.412206
3,1UDFq2InljM,0.097652,0.902348
4,1UDFq2InljM,0.443144,0.556856
...,...,...,...
460,zjd4HrJbc8o,0.217783,0.782217
461,zjd4HrJbc8o,0.578055,0.421945
462,zjd4HrJbc8o,0.659426,0.340574
463,zjd4HrJbc8o,0.737598,0.262402


In [ ]:
mean_pred_probs_2 = modelTrainLib.get_predClass_per_audio(pred_probs, label_dict)

and this would be the same as above:

In [ ]:
from classifier import GP_mini

In [ ]:
cls=GP_mini()

etc

## Evaluation

In [ ]:
import classifEvaluationFunctions as evalLib
evalLib.evaluation_Step(mean_pred_probs)
    